S-timator : demonstration of parameter estimation.
------------------------------------------

The **estimation.py** module combines ODE solving with the DE (differential evolution) genetic optimizer.


In [ ]:
%matplotlib inline

In [ ]:
from __future__ import print_function

In [ ]:
from stimator import read_model, read_tc

###  Parameter estimation: three reaction linear pathway

In [ ]:
mdl = """# Example file for S-timator
title Example 2

vin  : -> x1     , rate = k1
v2   : x1 ->  x2 , rate = k2 * x1
vout : x2 ->     , rate = k3 * x2

init : x1=0, x2=0

find k1 in [0, 2]
find k2 in [0, 2]
find k3 in [0, 2]

!! x1 x2

timecourse ../../examples/ex2data.txt
popsize = 60     # population size in GA
"""

print('----------- Model ------------------------')
print(mdl)

m1 = read_model(mdl)

print('----------- Time course -------------------')
print(read_tc(m1))

In [ ]:
best = m1.estimate()

print(best)
best.plot()

One can update the model parameters to the best fit values and obtain the same timecourse

In [ ]:
m2 = m1.copy()
bestpars = [(n,v) for n,v,e in best.parameters]
m2.update(bestpars)

m2.solve(tf=20.0).plot()

###  Parameter estimation: glyoxalase system

An example with **two time courses**

In [ ]:
mdl = """
title Glyoxalase system in L. Infantum

glx1 : HTA -> SDLTSH, V1*HTA/(Km1 + HTA)
glx2 : SDLTSH ->,     V2*SDLTSH/(Km2 + SDLTSH)

find V1  in [0.00001, 0.0001]
find Km1 in [0.01, 1]
find V2  in [0.00001, 0.0001]
find Km2 in [0.01, 1]

init : SDLTSH = 7.69231E-05, HTA = 0.1357
"""
m1 = read_model(mdl)
print(mdl)

best = m1.estimate(['TSH2a.txt', 'TSH2b.txt'], tc_dir='../../examples', names=['SDLTSH', 'HTA'], dump_generations=True)

print(best)

best.plot(fig_size=(14,6))
best.plot_generations(pars=['V2', 'Km1'], fig_size=(14,6))

-----------

An example with an **unknown initial value**

In [ ]:
m2 = m1.copy()

# Assume init.HTA is uncertain
m2.init.HTA.set_bounds((0.05,0.25))

# do not estimate Km1 and Km2, just to help the analysis
m2.parameters.Km1.set_bounds(None)
m2.parameters.Km2.set_bounds(None)
m2.parameters.Km1 = 0.252531
m2.parameters.Km2 = 0.0980973


# VERY IMPORTANT:
# only one time course can be used: 
# cannot fit one initial value using several timecourses!

best = m2.estimate(['TSH2a.txt'], names=['SDLTSH', 'HTA'], tc_dir='../../examples', opt_settings=dict(pop_size=60))

print(best)
best.plot()